In [1]:
# pip install -r ../requirements.txt

In [2]:
from utils import utils
from app import run_app, App
from core.gen_topics import gen_topics
from graphneo4j import GraphNeo4j
import warnings
warnings.filterwarnings("ignore") # supress all the warnigns




/Users/leonardoclo/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/multiclass.py:14: DeprecationWarning: Please use `spmatrix` from the `scipy.sparse` namespace, the `scipy.sparse.base` namespace is deprecated.
  from scipy.sparse.base import spmatrix
/Users/leonardoclo/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/optimize.py:18: DeprecationWarning: Please use `line_search_wolfe2` from the `scipy.optimize` namespace, the `scipy.optimize.linesearch` namespace is deprecated.
  from scipy.optimize.linesearch import line_search_wolfe2, line_search_wolfe1
/Users/leonardoclo/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/optimize.py:18: DeprecationWarning: Please use `line_search_wolfe1` from the `scipy.optimize` namespace, the `scipy.optimize.linesearch` namespace is deprecated.
  from scipy.optimize.linesearch import line_search_wolfe2, line_search_wolfe1


## Load the data from csv files

## Generate topics
  - Use this section to generate the csv's for topics and keywords

In [3]:
settings = utils.get_settings_notebook('../settings_topics.json')
app = App(settings)
dfs = app.extract_transform()

movie_topics = gen_topics()
# (df_movie_topics, df_topic_words) = movie_topics.batch_process(data_df['movies'][data_df['movies']['lang'] == 'en'])
(df_movie_topics, df_topic_words) = movie_topics.batch_process(dfs['movies'][
    dfs['movies']['overview'].apply(lambda x: x is not None and len(x) > 30)
])
df_topic_words.reset_index(drop=True, inplace=True)
df_movie_topics.reset_index(drop=True, inplace=True)
# display(df_topic_words)
# display(df_movie_topics)
df_topic_words.to_csv('./files/archive/topic_words.csv', index=True)
df_movie_topics.to_csv('./files/archive/movie_topic_ids.csv', index=True)

==== TRANSFORM directors ====
==== TRANSFORM movies ====
==== TRANSFORM cast ====


2022-12-12 00:45:24 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2022-12-12 00:45:24,863 : INFO : Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2022-12-12 00:45:26 INFO: Loading these models for language: en (English):
| Processor    | Package   |
----------------------------
| tokenize     | combined  |
| pos          | combined  |
| lemma        | combined  |
| depparse     | combined  |
| sentiment    | sstplus   |
| constituency | wsj       |
| ner          | ontonotes |

2022-12-12 00:45:26,710 : INFO : Loading these models for language: en (English):
| Processor    | Package   |
----------------------------
| tokenize     | combined  |
| pos          | combined  |
| lemma        | combined  |
| depparse     | combined  |
| sentiment    | sstplus   |
| constituency | wsj       |
| ner          | ontonotes |

2022-12-12 00:45:26 INFO: Use device: cpu
2022-12-12 00:45:26,712 : INFO : Use device: cpu
2022-12-12 00:45:26 INFO: Loading: tokenize
2022-12-12 00:45:26,713 : INFO : Loading: tokenize
2022-12-12 00:45:26 INFO: Loading: pos
2022-12-12 00:45:26,724 : INFO : Loading: pos
2022-12-12 00:45:27 INFO: Loading: lemma
2022-12

Failed at k = 0
    movie_id    imdb_id lang                                  title  \
0          2  tt0094675   fi                                  Ariel   
1          3  tt0092149   fi                    Shadows in Paradise   
2         12  tt0266543   en                           Finding Nemo   
3         13  tt0109830   en                           Forrest Gump   
4         14  tt0169547   en                        American Beauty   
5         15  tt0033467   en                           Citizen Kane   
6         16  tt0168629   en                     Dancer in the Dark   
7         18  tt0119116   en                      The Fifth Element   
8         19  tt0017136   de                             Metropolis   
9         24  tt0266697   en                      Kill Bill: Vol. 1   
10        25  tt0418763   en                                Jarhead   
11        28  tt0078788   en                         Apocalypse Now   
12        33  tt0105695   en                             Unfo

AttributeError: 'list' object has no attribute 'reset_index'

## Load the dataframes into graph database (Neo4j)

In [ ]:
settings = utils.get_settings_notebook('../settings.json')
app = App(settings)
app.etl()

## Display the final dataframe for the movies

In [ ]:
dfs['movies']

## ADD KEYWORDS NODE

In [ ]:
graph4j = GraphNeo4j(**{**settings['neo4j'], 'clear_data': False})

In [ ]:
qry = '''
    MATCH (t:Topic)<-[]-(m:Movie) UNWIND(t.topics) as keyword
    MERGE (k:Keyword {keyword: keyword})
    MERGE (k)<-[:HAS_KEYWORD]-(m)
'''
graph4j.run_query(qry, {})